In [1]:
import os
from base64 import b64decode
import openai
from datetime import datetime 
from tqdm.notebook import tqdm
from enum import Enum
import time
from glob import glob
from PIL import Image
import random

class WeatherType(Enum):
    cloudy = 1,
    foggy = 2,
    rainy =3 ,
    snowy = 4,
    sunny =5

base = "%s in %s weather"

locations = ["city place", "contry road", "city","county side","mountain","city road", "buildings","garden","river","forrest","sea", "island"]

weatherStrings = {
    WeatherType.cloudy:"cloudy",
    WeatherType.foggy:"foggy",
    WeatherType.rainy:"rainy",
    WeatherType.snowy:"snowy",
    WeatherType.sunny:"sunny",
}

In [2]:
def convert_and_save(data, file_path):
    image_data = b64decode(data)
    with open(file_path, mode="wb") as png:
        png.write(image_data)

#
# Promts
#

# generate_images("City place in rainy weather",1,outfolder="out")
def generate_images(promt, count = 1, api_key = os.getenv("OPENAI_API_KEY"), outfolder = "", size= "256x256"):
    openai.api_key = api_key
    response = openai.Image.create(
    prompt=promt,
    n=count,
    size=size,
    response_format="b64_json")
    for i,image in enumerate(response["data"]):
        convert_and_save(image["b64_json"],f"{outfolder}/{datetime.now().strftime('%Y%m%d%H%M%S')}_{i}.png")

#generate_promts(WeatherType.cloudy, count=24)
def generate_promts(weatherType: WeatherType, count= 10):
    images_per_loc = count //len(locations) 
    return [(base%(loc,weatherStrings[weatherType]),images_per_loc+1 if count % len(locations) > i else images_per_loc) for i,loc in enumerate(locations) if count > i]

def generate_images_for_type(weatherType: WeatherType, count = 1, api_key = os.getenv("OPENAI_API_KEY"), outfolder = "", size= "256x256"):
    os.makedirs(outfolder, exist_ok=True)
    prompts = generate_promts(weatherType=weatherType,count=count)
    for prompt, count_prompt in tqdm(prompts):
        success = False
        error_counter = 0
        while(not success):
            try:
                generate_images(promt=prompt,count=count_prompt,api_key = api_key,outfolder=outfolder,size=size)
                success = True
            except openai.error.OpenAIError as e:
                if error_counter > 5:
                    print(f"Aborting...")
                    return
                print(e.error,f"... Retrying in {2**error_counter} second(s)")
                time.sleep(2**error_counter)
                error_counter += 1

#
# Varations
#


#generate_variations(r"../data/kaggle-5-class-weather-dataset-resized/cloudy/cloudy-0.jpg",1,outfolder="out")
tempPath = 'temp.png'
def generate_variations(image_path, count = 1, api_key = os.getenv("OPENAI_API_KEY"), outfolder = "", size= "256x256"):
    Image.open(image_path).save(tempPath)
    openai.api_key = api_key
    response = openai.Image.create_variation(
    image=open(tempPath, "rb"),
    n=count,
    size=size,
    response_format="b64_json")
    os.remove(tempPath)
    for i,image in enumerate(response["data"]):
        convert_and_save(image["b64_json"],f"{outfolder}/{datetime.now().strftime('%Y%m%d%H%M%S')}_{i}.png")

def generate_varation_paths(input_folder,count= 10):
    image_paths = glob(f"{input_folder}/*")
    random.shuffle(image_paths)
    images_per_path = count // len(image_paths) 
    return [(img,images_per_path+1 if count % len(image_paths) > i else images_per_path) for i,img in enumerate(image_paths) if count > i]

def generate_varations_of_folder(input_folder, count = 1, api_key = os.getenv("OPENAI_API_KEY"), outfolder = "", size= "256x256"):
    os.makedirs(outfolder, exist_ok=True)
    prompts = generate_varation_paths(input_folder,count=count)
    for prompt, count_prompt in tqdm(prompts):
        success = False
        error_counter = 0
        while(not success):
            try:
                generate_variations(image_path=prompt,count=count_prompt,api_key = api_key,outfolder=outfolder,size=size)
                success = True
            except openai.error.OpenAIError as e:
                if error_counter > 5:
                    print(f"Aborting...")
                    return
                print(e.error,f"... Retrying in {2**error_counter} second(s)")
                time.sleep(2**error_counter)
                error_counter += 1


In [ ]:
api_key="<api key here>" # TODO

In [3]:
total_count = 400
count = total_count // len(WeatherType)
for x in tqdm(WeatherType,"Types"):
    generate_images_for_type(weatherType=x,count=count, outfolder=f"out/prompt400/{weatherStrings[x]}", api_key=api_key)

Types:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 1 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 2 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 4 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to l

  0%|          | 0/12 [00:00<?, ?it/s]

{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 1 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 2 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 4 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 52/1min. Please visit https://help.openai.com/en/articles/6696591 to l

  0%|          | 0/12 [00:00<?, ?it/s]

{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 53/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 1 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 53/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 2 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 53/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 4 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 53/1min. Please visit https://help.openai.com/en/articles/6696591 to l

  0%|          | 0/12 [00:00<?, ?it/s]

{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 55/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 1 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 55/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 2 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 55/1min. Please visit https://help.openai.com/en/articles/6696591 to learn how to increase your rate limit.",
  "param": null,
  "type": "requests"
} ... Retrying in 4 second(s)
{
  "code": null,
  "message": "Rate limit reached for images per minute. Limit: 50/1min. Current: 55/1min. Please visit https://help.openai.com/en/articles/6696591 to l

In [4]:
#path_base ="../data/kaggle-5-class-weather-dataset-resized/"
path_base ="../data/kaggle-rome-weather-dataset-resized/"
total_count = 400
count = total_count // len(WeatherType)
for x in tqdm(WeatherType,"Types"):
    path = f"{path_base}{weatherStrings[x]}"
    generate_varations_of_folder(input_folder=path,count=count, outfolder=f"out/variations_rome/{weatherStrings[x]}", api_key=api_key)

Types:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]